In [213]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [214]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

class AdaBoost:
    def __init__(self, n_estimators=10):
        self.n_estimators = n_estimators

    def calc_alpha(self, error):
        return 0.5 * np.log((1 - error) / (error + 1e-10))

    def sign(self, val):
        return 1 if val > 0 else 0

    def myFit(self, features, target):
        n_samples = features.shape[0]

        df = pd.DataFrame(features)
        df["target"] = target
        df["weights"] = 1 / n_samples

        self.alphas = []
        self.models = []

        for _ in range(self.n_estimators):
            dt = DecisionTreeClassifier(max_depth=1)

            dt.fit(features, target, sample_weight=df["weights"])
            pred = dt.predict(features)

            error = np.sum(df["weights"][df["target"] != pred])

            alpha = self.calc_alpha(error)

            df["weights"] *= np.exp(
                alpha * (df["target"] != pred).astype(int) -
                alpha * (df["target"] == pred).astype(int)
            )

            df["weights"] /= df["weights"].sum()

            self.models.append(dt)
            self.alphas.append(alpha)

    def myPredict(self, features):
        predictions = []

        for x in features:
            res = 0
            for alpha, model in zip(self.alphas, self.models):
                pred = model.predict(x.reshape(1, -1))[0]
                pred = 1 if pred == 1 else -1
                res += alpha * pred

            predictions.append(1 if res > 0 else 0)

        return np.array(predictions)


In [215]:
df = pd.read_csv("D://Programming//Machine_Learning//MachineLearningAlgo//SupervisedLearning//DataSets//Breast_cancer_dataset.csv")
target = df.iloc[:,1]
features = df.iloc[:,2:-1]
le = LabelEncoder()
target = le.fit_transform(target.values)
sc = StandardScaler()
features = sc.fit_transform(features)
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

In [216]:
from sklearn.metrics import accuracy_score
model = AdaBoost()
model.myFit(x_train,y_train)

In [217]:
y_pred = model.myPredict(x_test)
accuracy_score(y_pred,y_test)

0.9649122807017544